# IDAES-PSE and Reinforcement Learning (RL): HDA Example Notebook

The purpose of this notebook is to guide the user through the flow sheet design process. For this example, we will be designing a **hydrodealkylation (HDA)** process system, in which toluene reacts with hydrogen gas at high temperatures to form benzene. 

**C<sub>6</sub>H<sub>5</sub>CH<sub>3</sub> + H<sub>2</sub> &rarr; C<sub>6</sub>H<sub>6</sub> + CH<sub>4</sub>** 

In [28]:
import HDA_IDAES as HDAex
import HDA_IDAES_obs2list as HDAls
import RL_CORE as RL
import MasterControl_GUI as GUI

import ipywidgets as widgets 
from ipywidgets import AppLayout, Box, Button, GridspecLayout, Layout 
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

## Select Units

In [29]:
grid = GridspecLayout(9, 1)

unit_list = [
    'Heater',
    'Cooler',
    'Heat Exchanger',
    'Reactor',
    'Mixer',
    'Flash',
    'Splitter',
    'Compressor',
    'Expander'
    ]

for i in range(9):
    for j in range(1):
        grid[i, j] = widgets.Dropdown(
            options=['0', '1', '2'],
            value='1',
            description=unit_list[i],
            disabled=False,
            style = {'description_width': 'initial'}
)

In [30]:
header_button = widgets.HTML(
    value='<{size}>Select Max. Number of Units</{size}>'.format(size='h2'))

center_button = grid

In [31]:
AppLayout(header=header_button,
          left_sidebar=None,
          center=center_button,
          right_sidebar=None,
          footer=None)

AppLayout(children=(HTML(value='<h2>Select Max. Number of Units</h2>', layout=Layout(grid_area='header')), Gri…

In [32]:
num_heaters = int(grid[0,0].value)
num_coolers = int(grid[1, 0].value)
num_heatex = int(grid[2, 0].value)
num_reactors = int(grid[3, 0].value)
num_mixers = int(grid[4, 0].value)
num_flash = int(grid[5, 0].value)
num_splitters = int(grid[6, 0].value)
num_compressors = int(grid[7, 0].value)
num_expanders = int(grid[8, 0].value)

## RL Options

Select desired reinforcement learning parameters

First, how many episodes or training steps? (Default: 300,000)
Next, define the starting *e-greedy* value, or whether the model prioritizes **exploration** vs. **highest reward**. A value of 1 means the model only explores, whereas a value of 0 means the model only picks the highest reward. (Default: 0.8)
Then, select the *e-greedy increment* and learning rate and GNN option.

In [36]:
episodes_widget = widgets.IntSlider(
    value=300000,
    min=0,
    max=500000,
    step=50000,
    description='Episodes:',
    style = {'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

greedy_widget = widgets.FloatSlider(
    value=0.8,
    min=0,
    max=1,
    step=0.1,
    description='Greedy:',
    style = {'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

greedy_inc_widget = widgets.FloatSlider(
    value=5e-6,
    min=0,
    max=5e-3,
    step=5e-7,
    description='e-greedy increment:',
    style = {'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format= '.6f',
)

learning_widget = widgets.FloatSlider(
    value=0.01,
    min=0,
    max=0.1,
    step=0.01,
    description='Learning rate:',
    style = {'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format= '.2f',
)

GNN_widget = widgets.Dropdown(
    options=['True', 'False'],
    value='False',
    description= 'GNN Enabled',
    disabled=False,
    style = {'description_width': 'initial'}
)

display(episodes_widget, greedy_widget, greedy_inc_widget, learning_widget, GNN_widget)

IntSlider(value=300000, continuous_update=False, description='Episodes:', max=500000, step=50000, style=Slider…

FloatSlider(value=0.8, continuous_update=False, description='Greedy:', max=1.0, readout_format='.1f', style=Sl…

FloatSlider(value=5e-06, continuous_update=False, description='e-greedy increment:', max=0.005, readout_format…

FloatSlider(value=0.01, continuous_update=False, description='Learning rate:', max=0.1, step=0.01, style=Slide…

Dropdown(description='GNN Enabled', index=1, options=('True', 'False'), style=DescriptionStyle(description_wid…

In [37]:
episodes = episodes_widget.value
greedy = greedy_widget.value
greedy_inc = greedy_inc_widget.value
learning = learning_widget.value
GNN = GNN_widget.value

## Run RL

## Visualizations

In [54]:
file1 = open("result_n9/IDAES_unique_feasible_1.png", mode='rb')
image1 = file1.read()
file2 = open("result_n9/IDAES_unique_1.png", mode='rb')
image2 = file2.read()
cases = widgets.Image(
    value=image2,
    format='png',
    width=400,
    height=600,
)
feasible = widgets.Image(
    value=image1,
    format='png',
    width=400,
    height=600,
)

sidebyside = widgets.HBox([cases, feasible])
display.display(sidebyside)

* RL outputs
* **top 10 IDAES flow sheets and rerun**
* Dataframe with flow sheet specs/criteria
    * Overall Mass balance
    * Overall Energy balance
    * Net energy input (process heat input)
    * Yields
        * Hydrogen utilization
    * flow sheet differences
* **Plots**
    * Hydrogen utilization
    * **Cost**
    * Show benefits
        * thermal integration
            * heat wasted
            * pre-heating of inputs
   